In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datashader as ds
import datashader.transfer_functions as tf
from datashader.mpl_ext import dsshow
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Load data
completed_dt = pd.read_csv('../data/nb.csv')
order_dt = pd.read_csv('../data/driver_locations_during_request.csv')

# Basic statistics
print("Basic Statistics:\n", completed_dt.describe())
print("basic statistics of \n", order_dt.describe())


# Missing values
print("Missing Values:\n", completed_dt.isnull().sum())
print("Missing values: \n", order_dt.isnull().sum())

print("completed data head \n", completed_dt.head())
print("total order data head \n", order_dt.head())


completed_dt['Trip Start Time']=pd.to_datetime(completed_dt['Trip Start Time'])
completed_dt['Trip End Time']=pd.to_datetime(completed_dt['Trip End Time'])
completed_dt['trip duration'] = completed_dt['Trip End Time'] - completed_dt['Trip Start Time']
print("the following is completed data with trip duration less than zero \n", completed_dt["trip duration"]<=0)
# Outlier detection (example: using Z-score)
z_scores = np.abs((completed_dt - completed_dt.mean()) / completed_dt.std())
completed_dt = completed_dt[(z_scores < 3).all(axis=1)]

# # Treat missing values (example: fill with median)
# completed_dt.fillna(completed_dt.mean(), inplace=True)

# # Basic visualizations
# plt.figure(figsize=(10, 6))
# sns.histplot(completed_dt['order_location_latitude'], kde=True)
# plt.title('Distribution of Order Location Latitude')
# plt.show()

# plt.figure(figsize=(10, 6))
# sns.histplot(completed_dt['order_location_longitude'], kde=True)
# plt.title('Distribution of Order Location Longitude')
# plt.show()
s
# # Geospatial Visualization using Datashader
# canvas = ds.Canvas(plot_width=400, plot_height=400)
# agg = canvas.points(completed_dt, 'order_location_longitude', 'order_location_latitude')
# img = tf.shade(agg)
# tf.set_background(img, "black").to_pil().show()

# # More detailed visualizations and insights
# # Correlation heatmap
# plt.figure(figsize=(12, 8))
# sns.heatmap(completed_dt.corr(), annot=True, cmap='coolwarm')
# plt.title('Correlation Heatmap')
# plt.show()

# # Save cleaned data
# completed_dt.to_csv('data/cleaned_geospatial_data.csv', index=False)


Basic Statistics:
             Trip ID
count  5.360200e+05
mean   9.910221e+05
std    3.456364e+05
min    3.919960e+05
25%    6.935610e+05
50%    9.910085e+05
75%    1.281570e+06
max    1.637709e+06
basic statistics of 
                  id      order_id     driver_id           lat           lng  \
count  1.557740e+06  1.557740e+06  1.557740e+06  1.557740e+06  1.557740e+06   
mean   7.788705e+05  4.191833e+05  2.341505e+05  6.536031e+00  3.378877e+00   
std    4.496809e+05  1.444531e+04  2.270475e+04  5.998150e-02  5.333432e-02   
min    1.000000e+00  3.920010e+05  1.219810e+05  6.409333e+00  3.076561e+00   
25%    3.894358e+05  4.067650e+05  2.429970e+05  6.498694e+00  3.348810e+00   
50%    7.788705e+05  4.216870e+05  2.435890e+05  6.544247e+00  3.363504e+00   
75%    1.168305e+06  4.293130e+05  2.440560e+05  6.593276e+00  3.385177e+00   
max    1.557740e+06  5.179480e+05  2.478770e+05  7.702536e+00  8.515414e+00   

       created_at  updated_at  
count         0.0         0.0  
mea

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [2]:
# Additional feature creation based on time and location
completed_dt['Trip Start Time'] = pd.to_datetime(completed_dt['Trip Start Time'])
completed_dt['end_time'] = pd.to_datetime(completed_dt['end_time'])

# Extracting features from datetime
completed_dt['hour_of_day'] = completed_dt['start_time'].dt.hour
completed_dt['day_of_week'] = completed_dt['start_time'].dt.dayofweek
completed_dt['is_weekend'] = completed_dt['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

# Placeholder for external data features (rain, holiday, traffic)
# You would need to merge external data sources for actual implementation
completed_dt['rain'] = np.random.choice([0, 1], size=len(completed_dt))  # Placeholder
completed_dt['holiday'] = np.random.choice([0, 1], size=len(completed_dt))  # Placeholder
completed_dt['traffic_condition'] = np.random.choice(['low', 'medium', 'high'], size=len(completed_dt))  # Placeholder

# One-hot encode categorical features
completed_dt = pd.get_dummies(completed_dt, columns=['traffic_condition'], drop_first=True)

# Feature extraction example - travel time
df['travel_time'] = (df['end_time'] - df['start_time']).dt.total_seconds() / 60  # in minutes

print("Head of dataframe with new features:\n", df.head())

# Save the dataframe with new features
completed_dt.to_csv('data/enriched_geospatial_data.csv', index=False)


NameError: name 'df' is not defined

In [3]:
# Feature scaling
scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[['order_location_latitude', 'order_location_longitude', 'travel_time']] = scaler.fit_transform(df_scaled[['order_location_latitude', 'order_location_longitude', 'travel_time']])

print("Head of scaled dataframe:\n", df_scaled.head())

# Save the scaled dataframe
df_scaled.to_csv('data/scaled_geospatial_data.csv', index=False)


NameError: name 'df' is not defined

In [4]:
from geopy.distance import geodesic

def compute_distance(row):
    start_coords = (row['driver_location_latitude'], row['driver_location_longitude'])
    end_coords = (row['order_destination_latitude'], row['order_destination_longitude'])
    return geodesic(start_coords, end_coords).meters

df['distance_m'] = df.apply(compute_distance, axis=1)

# Compute driving speed (placeholder example)
df['driving_speed'] = df['distance_m'] / df['travel_time']  # in meters per minute

print("Head of dataframe with distance and speed:\n", df.head())

# Save the dataframe with distance and speed
df.to_csv('data/feature_enriched_geospatial_data.csv', index=False)


ModuleNotFoundError: No module named 'geopy'

In [5]:
from scipy.spatial import KDTree

# Create KDTree for efficient radius searches
locations = df[['order_location_latitude', 'order_location_longitude']].values
kdtree = KDTree(locations)

def count_neighbors_within_radius(row, radius=500):
    point = (row['order_location_latitude'], row['order_location_longitude'])
    indices = kdtree.query_ball_point(point, radius / 1000)  # radius in km
    return len(indices)

df['riders_within_500m'] = df.apply(count_neighbors_within_radius, axis=1)

print("Head of dataframe with riders within 500m:\n", df.head())

# Save the dataframe with riders count
df.to_csv('data/riders_enriched_geospatial_data.csv', index=False)


NameError: name 'df' is not defined

In [6]:
from sklearn.cluster import KMeans

# Clustering starting locations
kmeans_start = KMeans(n_clusters=5)
df['start_location_cluster'] = kmeans_start.fit_predict(df[['driver_location_latitude', 'driver_location_longitude']])

# Clustering destination locations
kmeans_dest = KMeans(n_clusters=5)
df['destination_location_cluster'] = kmeans_dest.fit_predict(df[['order_destination_latitude', 'order_destination_longitude']])

print("Head of dataframe with clusters:\n", df.head())

# Save the dataframe with clusters
df.to_csv('data/clustered_geospatial_data.csv', index=False)


NameError: name 'df' is not defined

In [7]:
# Purpose-driven visualization using Datashader
canvas = ds.Canvas(plot_width=800, plot_height=800)
agg = canvas.points(df, 'order_location_longitude', 'order_location_latitude')
img = tf.shade(agg)
tf.set_background(img, "black").to_pil().show()

# Cluster visualization
cluster_canvas = ds.Canvas(plot_width=800, plot_height=800)
agg = cluster_canvas.points(df, 'order_location_longitude', 'order_location_latitude', ds.count_cat('start_location_cluster'))
img = tf.shade(agg)
tf.set_background(img, "black").to_pil().show()


NameError: name 'df' is not defined